In [2]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results/execute_ils.csv')


In [3]:
df.head()

,Instance,Algorithme,Seed,Score,NbPertubations,MaxEval,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,50,1879,6,100000,43.475
1,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,50,1971,9,100000,50.171
2,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,51,2014,6,100000,40.724
3,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,51,2049,9,100000,47.291
4,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,52,1896,6,100000,48.810


In [10]:
neighbor_relations = ["swap","2opt"]
choice_mehod = ['first', 'best']
versus = [('first', 'best')]
instances = df['Instance'].unique()
nb_perturbs = df["NbPertubations"].unique()

In [15]:
def get_score(instance,neighbor_relation,choice_mehod,seed,nbp):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"iterated_local_search_{choice_mehod}_improver_{neighbor_relation}") &
    (df['Seed'] == seed) & 
    (df["NbPertubations"] == nbp)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,choice_mehod,seed)
    

In [21]:
def gagne(instance,neighbor_relation,seed, vs,nbp):
    #print("vs:",vs)
    s1 = get_score(instance,neighbor_relation,vs[0],seed,nbp)
    s2 = get_score(instance,neighbor_relation,vs[1],seed,nbp)
    return s1 < s2

In [20]:
get_score("instances/atsp_rand_50_75.txt","2opt","best",50,9)

1971

In [26]:
get_score("instances/atsp_rand_50_75.txt","2opt","first",50,9)

1860

In [28]:
gagne("instances/atsp_rand_50_75.txt","2opt",50, ("first","best"),9)

True

In [34]:
result_tuple={"instances" : instances}
for instance in instances:
    #print(instance)
    for nr in neighbor_relations:
        #print(nr)
        for vs in versus:
            #print(vs," : ",end="")
            r=[0,0]
            for nbp in nb_perturbs:
                for seed in df["Seed"].unique():
                    if gagne(instance,nr,seed, vs,nbp):
                        r[0]+=1
                    else:
                        r[1]+=1
            try:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"].append((r[0],r[1]))
            except:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"] = [(r[0],r[1])]
            #print(r)
                
result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_first_vs_best,2opt_first_vs_best
0,instances/atsp_rand_50_75.txt,"(11, 9)","(16, 4)"
1,instances/atsp_rand_50_100.txt,"(10, 10)","(19, 1)"
2,instances/atsp_rand_60_50.txt,"(13, 7)","(17, 3)"
3,instances/atsp_rand_60_100.txt,"(5, 15)","(15, 5)"
4,instances/atsp_rand_70_55.txt,"(14, 6)","(16, 4)"
5,instances/atsp_rand_70_100.txt,"(13, 7)","(19, 1)"
6,instances/atsp_rand_80_70.txt,"(14, 6)","(18, 2)"
7,instances/atsp_rand_80_100.txt,"(19, 1)","(17, 3)"


In [36]:
result={"instances" : instances}
for instance in instances:
    #print(instance)
    for nr in neighbor_relations:
        #print(nr)
        for vs in versus:
            #print(vs," : ",end="")
            r=[0,0]
            for nbp in nb_perturbs:
                for seed in df["Seed"].unique():
                    if gagne(instance,nr,seed, vs,nbp):
                        r[0]+=1
                    else:
                        r[1]+=1
            try:
                result[f"{nr}_{vs[0]}_vs_{vs[1]}"].append((r[0]))
            except:
                result[f"{nr}_{vs[0]}_vs_{vs[1]}"] = [(r[0])]
            #print(r)


result = pd.DataFrame(result)
result

,instances,swap_first_vs_best,2opt_first_vs_best
0,instances/atsp_rand_50_75.txt,11,16
1,instances/atsp_rand_50_100.txt,10,19
2,instances/atsp_rand_60_50.txt,13,17
3,instances/atsp_rand_60_100.txt,5,15
4,instances/atsp_rand_70_55.txt,14,16
5,instances/atsp_rand_70_100.txt,13,19
6,instances/atsp_rand_80_70.txt,14,18
7,instances/atsp_rand_80_100.txt,19,17


In [38]:
mean_swap_first = result["swap_first_vs_best"].sum()/8
mean_swap_first

12.375

In [39]:
mean_swap_best = (8*20 -result["swap_first_vs_best"].sum() ) /8
mean_swap_best

7.625

In [40]:
mean_2opt_first = result["2opt_first_vs_best"].sum()/8
mean_2opt_first

17.125

In [41]:
mean_2opt_best = (8*20 -result["2opt_first_vs_best"].sum() ) /8
mean_2opt_best

2.875